In [ ]:
import sys
from pathlib import Path
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))

In [ ]:
import pandas as pd
import numpy as np
from os.path import join

from datetime import datetime, date, timedelta
import random



from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

random.seed(10)

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)

cc.df = pd.merge(cc.df.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','macro','rs','mun_resid','uf','municipio','regional','populacao']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
cc.df = pd.merge(cc.df.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

In [ ]:
casos_confirmados = cc.df.loc[(cc.df['data_diagnostico'] >= '2021-07-01') & (cc.df['data_diagnostico'] <= '2021-07-14')]
casos_confirmados = casos_confirmados.loc[casos_confirmados['uf'] == 'PR']

In [ ]:
casos_confirmados.groupby(['macro', 'rs', 'municipio','populacao'])[['sexo']].count().reset_index()

In [ ]:
coef_inc = casos_confirmados.groupby(['macro', 'rs', 'municipio','populacao'])[['sexo']].count().reset_index().rename(columns={'sexo':'casos'})
coef_inc

In [ ]:
coef_inc['coef_incidencia'] = coef_inc['casos'] / coef_inc['casos'].sum()

In [ ]:
coef_inc['CAIXAS'] = 1

In [ ]:
coef_inc['CAIXAS_ADICIONAIS'] = np.floor(coef_inc['coef_incidencia'] * 1609)

In [ ]:
coef_inc['CAIXAS_ADICIONAIS'].sum()

In [ ]:
coef_inc['CAIXAS'] = coef_inc['CAIXAS'] + coef_inc['CAIXAS_ADICIONAIS'].fillna(0)
del(coef_inc['CAIXAS_ADICIONAIS'])
coef_inc

In [ ]:
pesos = coef_inc.loc[coef_inc['casos'] >= 50].reset_index()
pesos

In [ ]:
testes_randomicos = random.choices(pesos['municipio'], weights=pesos['coef_incidencia'], k=195)

In [ ]:
testes_previstos = coef_inc.join(pd.Series(testes_randomicos, name='CAIXAS_ADICIONAIS').value_counts(), on='municipio')
testes_previstos

In [ ]:
testes_previstos['CAIXAS'] = testes_previstos['CAIXAS'] + testes_previstos['CAIXAS_ADICIONAIS'].fillna(0)
del(testes_previstos['CAIXAS_ADICIONAIS'])
testes_previstos

In [ ]:
testes_previstos['CAIXAS'] = testes_previstos['CAIXAS'].fillna(0)
testes_previstos['TESTES'] = testes_previstos['CAIXAS'] * 20
testes_previstos.loc['TOTAL', ['populacao', 'casos', 'CAIXAS', 'TESTES']] = testes_previstos[['populacao', 'casos', 'CAIXAS', 'TESTES']].sum()

In [ ]:
testes_previstos

In [ ]:
testes_previstos = testes_previstos.sort_values(['rs', 'municipio'])
testes_previstos

In [ ]:
# POR MUNICIPIO

In [ ]:
testes_municipios = testes_previstos.rename(columns={'populacao':'POPULAÇÃO RESIDENTE',
                                'casos':'CASOS NOS ÚLTIMOS 3 MESES',
                                'coef_incidencia':'% do TOTAL de casos',
                                'CAIXAS':'NÚMERO DE CAIXAS',
                                'TESTES':'NÚMERO DE TESTES PREVISTOS',
                                'macro':'MACRO',
                                'municipio':'MUNICÍPIO',
                                'rs':'RS'})
testes_municipios.to_excel('testes_previstos_municipios.xlsx')

In [ ]:
testes_rs = testes_previstos.groupby('rs').sum()
testes_rs.loc['TOTAL', ['populacao', 'casos', 'CAIXAS', 'TESTES']] = testes_rs[['populacao', 'casos', 'CAIXAS', 'TESTES']].sum()

In [ ]:
# POR RS

In [ ]:
testes_rs = testes_rs.rename(columns={'populacao':'POPULAÇÃO RESIDENTE',
                                'casos':'CASOS NOS ÚLTIMOS 3 MESES',
                                'coef_incidencia':'% do TOTAL de casos',
                                'CAIXAS':'NÚMERO DE CAIXAS',
                                'TESTES':'NÚMERO DE TESTES PREVISTOS',
                                'macro':'MACRO',
                                'municipio':'MUNICÍPIO',
                                'rs':'RS'})
testes_rs.to_excel('testes_previstos_rs.xlsx')

In [ ]:
# POR MACRO

In [ ]:
testes_macro = testes_previstos.groupby('macro').sum()
testes_macro.loc['TOTAL', ['populacao', 'casos', 'CAIXAS', 'TESTES']] = testes_macro[['populacao', 'casos', 'CAIXAS', 'TESTES']].sum()

In [ ]:
testes_macro = testes_macro.rename(columns={'populacao':'POPULAÇÃO RESIDENTE',
                                'casos':'CASOS NOS ÚLTIMOS 3 MESES',
                                'coef_incidencia':'% do TOTAL de casos',
                                'CAIXAS':'NÚMERO DE CAIXAS',
                                'TESTES':'NÚMERO DE TESTES PREVISTOS',
                                'macro':'MACRO',
                                'municipio':'MUNICÍPIO',
                                'rs':'RS'})
testes_macro.to_excel('testes_previstos_macro.xlsx')